In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Stroke Prediction

## 1. Problem definition

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df.head()

In [ ]:
df.describe().astype(int)

In [ ]:
df.isna().sum()

In [ ]:
df.info()


In [ ]:
print('ever_married: ',df.ever_married.value_counts(), '\n')
print('gender: ',df.gender.value_counts(), '\n')
print('work_type: ',df.work_type.value_counts(), '\n')
print('Residence_type: ',df.Residence_type.value_counts(), '\n')
print('smoking_status: ',df.smoking_status.value_counts(), '\n')


In [ ]:
## Convert to numerical value

def to_cat(df):
    for label,content in df.items():
        if not pd.api.types.is_numeric_dtype(content):
            df[label] = content.astype('category')
            
def turn_to_numeric(df):
      for label,content in df.items():
        if not pd.api.types.is_numeric_dtype(content):
            df[label] = pd.Categorical(content).codes +1
    
to_cat(df)
turn_to_numeric(df)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
## Fill Nan
df.bmi = df.bmi.fillna(df.bmi.median())


# Exploiratory Data Analysis

In [ ]:
plt.figure(figsize=(16,12))
sns.heatmap(data=df.corr(), annot=True, cbar=None, cmap='YlGnBu')
plt.xticks(rotation=0)
plt.show()


* Stroke dependent mostly on age, hypertension, heart_disease, awg_glucose_level, and ever_married. 

In [ ]:
plt.figure(figsize=(12,8))
pd.crosstab(df.gender,df.stroke).plot.bar(color=['lightblue', 'red'])
plt.xticks(rotation=0)
plt.xlabel('1: Male, 2: Female, 3: Other')
plt.show()

In [ ]:
df.heart_disease.value_counts().plot.bar()

In [ ]:
pd.crosstab(df.heart_disease, df.stroke).plot.bar()
heart_no_stroke, heart_stroke = df.stroke[df.heart_disease==1].value_counts()
no_heart_no_stroke, no_heart_stroke = df.stroke[df.heart_disease==0].value_counts()
print(f'No heart disease: {(no_heart_stroke / (no_heart_stroke + no_heart_no_stroke))}')
print(f'Heart disease: {(heart_stroke / (heart_stroke + heart_no_stroke))}')

In [ ]:
plt.figure(figsize=(12,8))
sns.histplot(df.age, kde=True, bins=20)

In [ ]:
plt.scatter(df.age[df.stroke == 0], df.bmi[df.stroke==0], c='lightblue' )
plt.scatter(df.age[df.stroke == 1], df.bmi[df.stroke==1], c='red')
plt.legend(['No stroke', 'Stroke'])
plt.xlabel('Age')
plt.ylabel('BMI')

* age < 40, bmi + when age +. a > 50 bmi -, age +.
* Most of cases is > 50. BMI < 40.



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop('stroke', axis=1)
y = df.stroke

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
clf = RandomForestClassifier(n_estimators=100,random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)